In [ ]:
import os
os.chdir("C:\\Users\\erin.melly\\Desktop\\GitHub\\Tax-Calculator")
from behresp import *
from taxcalc import *
import pandas as pd
import numpy as np

In [ ]:
# dictionary of assumed elasticity each year
cg= {2021: -4.184, 2022: -3.488, 2023: -2.76, 
     2024: -2.76, 2025: -2.76, 2026: -2.76, 
     2027: -2.76, 2028: -2.76, 2029: -2.76, 
     2030: -2.76
    }

In [ ]:
def create_biden_distribution_table(cyr, taxref, group):
    assert taxref in ['individual', 'individual_business']
    assert group in ['standard_income_bins', 'weighted_deciles']
    assert cyr in range(2021, 2031)
    if taxref=='individual':
        # baseline
        rec = Records()
        pol = Policy()
        calc_base=Calculator(pol, rec)
        calc_base.advance_to_year(cyr)
        calc_base.calc_all()
        #reform
        rec = Records()
        pol = Policy()
        pol.implement_reform(Policy.read_json_reform('taxcalc\\reforms\\Biden.json'))
        calc_ref= Calculator(pol, rec)
        calc_ref.advance_to_year(cyr)
        calc_ref.calc_all()
    else:
        # baseline
        rec = Records()
        pol = Policy()
        baseline={
            "Business_Effect_Table_expinc": {cyr: True}
        }
        pol.implement_reform(baseline)
        calc_base= Calculator(pol, rec)
        calc_base.advance_to_year(cyr)
        calc_base.calc_all()
        # reform
        rec = Records()
        pol = Policy()
        pol.implement_reform(Policy.read_json_reform('taxcalc\\reforms\\Biden_with_business.json'))
        calc_ref= Calculator(pol, rec)
        calc_ref.advance_to_year(cyr)
        calc_ref.calc_all()   
    # implement behavioral responses
    response_elasticities = {'sub': 0.25, 'cg': cg[cyr]}
    base_df, ref_df = response(calc_base, calc_ref, response_elasticities, chg_inc=True)
    # create difference table (1)
    vdf1 = base_df[DIFF_VARIABLES]
    vdf2 = ref_df[DIFF_VARIABLES]
    distdf1 = create_difference_table(vdf1, vdf2, groupby=group, tax_to_diff='combined', pop_quantiles=False)
    # create difference table (2) [only use the pc_aftertaxinc column from this distribution table]
    response_elasticities = {'sub': 0.25, 'cg': cg[cyr]}
    base_df, ref_df = response(calc_base, calc_ref, response_elasticities, chg_inc=False)
    vdf1 = base_df[DIFF_VARIABLES]
    vdf2 = ref_df[DIFF_VARIABLES]
    distdf2 = create_difference_table(vdf1, vdf2, groupby=group, tax_to_diff='combined', pop_quantiles=False)
    # append pc_aftertaxinc from dataframe 2 and delete unused columns
    df = distdf1.assign(pch_chg_inc = distdf2['pc_aftertaxinc'])
    table = df.drop(['ubi', 'benefit_cost_total', 'benefit_value_total', 'pc_aftertaxinc', 'count', 'tax_cut',
                    'perc_cut', 'tax_inc', 'perc_inc', 'tot_change'], axis=1)
    # final table, by standard income bin
    return table

#### TABLE 3: TOTAL BURDEN: INDIVIDUAL AND BUSINESS REFORMS

In [ ]:
create_biden_distribution_table(cyr=2021, taxref='individual_business', group='weighted_deciles')

In [ ]:
create_biden_distribution_table(cyr=2030, taxref='individual_business', group='weighted_deciles')


#### INDIVIDUAL REFORMS

In [ ]:
create_biden_distribution_table(cyr=2021, taxref='individual', group='weigted_deciles')

In [ ]:
create_biden_distribution_table(cyr=2030, taxref='individual', group='weigted_deciles')